In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier

In [2]:
df=pd.read_csv(r"C:\Users\tareq\Desktop\p2\Codes\All_Users_Labeled.csv")

In [3]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.drop('Username',axis=1,inplace=True)

In [4]:
df

,Full_Text,total_words,total_char,avg_chr_per_word,total_emojis,o,c,e,a,n
0,كلمه انثي صارت تضرب عصبي بمناسبه المراه العالم...,2530.0,17071.0,6.087708,151.0,0,0,1,1,0
1,فش موظفين القهوه الكيوت اللي بحكيلك بنفع يحط ش...,1394.0,8593.0,5.656274,28.0,1,0,0,0,0
2,مباشر القداس الالهي وصلاه المراثي كنيسه القديس...,5404.0,38315.0,6.158481,2.0,0,1,0,1,0
3,ارتفع قهوه ارتفع قلبين قطعه شوكولاته ارتفع قل...,674.0,4665.0,6.584828,48.0,1,0,0,0,0
4,يتدحرج الارض الضحك فن غيم غيم وجه متعب ...,328.0,2398.0,6.471875,47.0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1643,اللهم عمره اعيد شتات قلبي الكعبه يقاس رقي الان...,2436.0,14009.0,4.763297,11.0,0,0,0,1,0
1644,تفعيل مهارات الرسم الهندسي لترسيخ مفهوم التماث...,3887.0,26702.0,5.747843,248.0,0,0,0,1,0
1645,تنهد نار نار يتدحرج الارض الضحك انتصار وجه...,646.0,3501.0,4.825641,75.0,1,0,0,0,0
1646,البعض يبقي اجلك يبقي حاجته وجه بالدوار وجه ا...,26.0,136.0,4.161111,2.0,1,0,0,0,0


In [5]:
X_num = df.iloc[:, 1:5].values
ss=StandardScaler()
X_num=ss.fit_transform(X=X_num)
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(df['Full_Text'])
df1=pd.DataFrame(X_text.toarray(),columns=vectorizer.get_feature_names_out())
df1['total_words']= [x[0] for x in X_num]
df1[ "total_char"]= [x[1] for x in X_num]
df1["avg_chr_per_word"]= [x[2] for x in X_num]
df1["total_emojis"]= [x[3] for x in X_num]

In [5]:
y = df['o'].values

In [69]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Concatenate,Dropout,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam


X = df['Full_Text']
#y = df[['o', 'c', 'e', 'a', 'n']]
y=df['n']
other_features = df[['total_words', 'total_char', 'avg_chr_per_word', 'total_emojis']]

X_train, X_test, y_train, y_test, other_features_train, other_features_test = train_test_split(X, y, other_features, test_size=0.2, random_state=42)

# Tokenize the input texts
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=95000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the input sequences
max_len = 2000 # Set a max length that captures most of the context without introducing too much padding
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

# Convert labels to categorical format
#y_train_cat = to_categorical(y_train)
#y_test_cat = to_categorical(y_test)

# Define the other features input layer
other_features_input = Input(shape=(4,), name='other_features_input')

# Define the text input layer
text_input = Input(shape=(max_len,), name='text_input')

# Define the embedding layer for the text input
embedding_dim = 128
embedded_text = Embedding(input_dim=175277, output_dim=embedding_dim, input_length=max_len)(text_input)

# Define the LSTM layer
lstm = LSTM(units=16)(embedded_text)
# Add Dense layer to reshape input_features
dense = Dense(units=16)(other_features_input)

# Concatenate the LSTM output and dense layer output
concatenated = Concatenate()([lstm, dense])

# Define the output layer

output = Dense(units=1, activation='sigmoid')(concatenated)

# Define the model with both the text input and other features input
model = Model(inputs=[text_input, other_features_input], outputs=output)

# compute class weights
total = y.shape[0]
class_0 = np.sum(y == 0)
class_1 = np.sum(y == 1)
w0 = total / class_0
w1 = total / class_1
class_weight = {0: w0, 1: w1}

# Compile the model
from keras.optimizers import SGD
opt = SGD(learning_rate=10, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = "binary_crossentropy", optimizer = 'adam',metrics=['accuracy'])

# Train the model
es = EarlyStopping(patience=3, verbose=1)
history = model.fit(
    x=[X_train_padded, other_features_train],
    y=y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    #shuffle=True,
    class_weight=class_weight,
    callbacks=[es]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x=[X_test_padded, other_features_test], y=y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')

Epoch 1/20
17/17 [==============================] - 32s 2s/step - loss: 944.7569 - accuracy: 0.9345 - val_loss: 26.7165 - val_accuracy: 0.9470
Epoch 2/20
17/17 [==============================] - 30s 2s/step - loss: 194.6606 - accuracy: 0.6262 - val_loss: 138.8735 - val_accuracy: 0.0530
Epoch 3/20
17/17 [==============================] - 30s 2s/step - loss: 60.3844 - accuracy: 0.5930 - val_loss: 11.9525 - val_accuracy: 0.3636
Epoch 4/20
17/17 [==============================] - 36s 2s/step - loss: 27.8611 - accuracy: 0.4241 - val_loss: 2.1007 - val_accuracy: 0.7235
Epoch 5/20
17/17 [==============================] - 42s 2s/step - loss: 19.8157 - accuracy: 0.5825 - val_loss: 10.4312 - val_accuracy: 0.3371
Epoch 6/20
17/17 [==============================] - 32s 2s/step - loss: 19.1174 - accuracy: 0.5446 - val_loss: 11.0839 - val_accuracy: 0.2765
Epoch 7/20
17/17 [==============================] - 31s 2s/step - loss: 14.4393 - accuracy: 0.5474 - val_loss: 2.4525 - val_accuracy: 0.6098
Epoch

In [ ]:
y_pred=model.predict(x=[X_test_padded, other_features_test])
y_pred=y_pred>0.3
print(classification_report(y_test,y_pred))

In [56]:
model.save('Final_models/rnn_o')

INFO:tensorflow:Assets written to: Final_models/rnn_o\assets


INFO:tensorflow:Assets written to: Final_models/rnn_o\assets


In [132]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Concatenate,Dropout,Flatten,
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam


X = df['Full_Text']
#y = df[['o', 'c', 'e', 'a', 'n']]
y=df['c']
other_features = df[['total_words', 'total_char', 'avg_chr_per_word', 'total_emojis']]

X_train, X_test, y_train, y_test, other_features_train, other_features_test = train_test_split(X, y, other_features, test_size=0.2, random_state=42)

# Tokenize the input texts
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=95000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the input sequences
max_len = 2000 # Set a max length that captures most of the context without introducing too much padding
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

# Convert labels to categorical format
#y_train_cat = to_categorical(y_train)
#y_test_cat = to_categorical(y_test)

# Define the other features input layer
other_features_input = Input(shape=(4,), name='other_features_input')

# Define the text input layer
text_input = Input(shape=(max_len,), name='text_input')

# Define the embedding layer for the text input
embedding_dim = 128
embedded_text = Embedding(input_dim=175277, output_dim=embedding_dim, input_length=max_len)(text_input)

# Define the LSTM layer
lstm = LSTM(units=8)(embedded_text)
# Add Dense layer 8 reshape input_features
dense = Dense(units=16)(other_features_input)

# Concatenate the LSTM output and dense layer output
concatenated = Concatenate()([lstm, dense])

# Define the output layer

output = Dense(units=1, activation='sigmoid')(concatenated)

# Define the model with both the text input and other features input
model = Model(inputs=[text_input, other_features_input], outputs=output)

# compute class weights
total = y.shape[0]
class_0 = np.sum(y == 0)
class_1 = np.sum(y == 1)
w0 = total / class_0
w1 = total / class_1
class_weight = {0: w0, 1: w1}

# Compile the model
from keras.optimizers import SGD
opt = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss = "binary_crossentropy", optimizer = 'adam',metrics=['accuracy'])

# Train the model
es = EarlyStopping(patience=3, verbose=1)
history = model.fit(
    x=[X_train_padded, other_features_train],
    y=y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    #shuffle=True,
    #class_weight=class_weight,
    callbacks=[es]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x=[X_test_padded, other_features_test], y=y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')

Epoch 1/20
17/17 [==============================] - 37s 2s/step - loss: 1493.1764 - accuracy: 0.4317 - val_loss: 1085.8458 - val_accuracy: 0.4091
Epoch 2/20
17/17 [==============================] - 34s 2s/step - loss: 616.7061 - accuracy: 0.4326 - val_loss: 169.4843 - val_accuracy: 0.4129
Epoch 3/20
17/17 [==============================] - 33s 2s/step - loss: 96.5139 - accuracy: 0.5465 - val_loss: 71.0002 - val_accuracy: 0.5909
Epoch 4/20
17/17 [==============================] - 34s 2s/step - loss: 34.3686 - accuracy: 0.5370 - val_loss: 27.5038 - val_accuracy: 0.5909
Epoch 5/20
17/17 [==============================] - 34s 2s/step - loss: 18.1529 - accuracy: 0.5550 - val_loss: 15.1530 - val_accuracy: 0.4811
Epoch 6/20
17/17 [==============================] - 31s 2s/step - loss: 15.0250 - accuracy: 0.5342 - val_loss: 13.7101 - val_accuracy: 0.4773
Epoch 7/20
17/17 [==============================] - 32s 2s/step - loss: 12.4025 - accuracy: 0.5171 - val_loss: 12.3087 - val_accuracy: 0.4811


In [174]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Dropout, BatchNormalization
from keras import regularizers, callbacks
import nlpaug.augmenter.word as naw

# load dataset
data = df.copy()

# split dataset into training, validation, and testing sets
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# data augmentation using nlpaug library
aug = naw.SynonymAug(aug_src='wordnet')

train_aug_data = train_data.copy()
for i in range(10):
    train_aug_data['Full_Text'] = train_aug_data['Full_Text'].apply(lambda x: aug.augment(x))
train_aug_data = train_aug_data.sample(frac=1)

# combine augmented data with original training data
train_data = pd.concat([train_data, train_aug_data], ignore_index=True)

# tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['Full_Text'])
X_train = tokenizer.texts_to_sequences(train_data['Full_Text'])
X_val = tokenizer.texts_to_sequences(val_data['Full_Text'])
X_test = tokenizer.texts_to_sequences(test_data['Full_Text'])

# pad sequences to make them of the same length
maxlen = 100  # max length of a text sequence
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tareq\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


Epoch 1/20
42/42 [==============================] - 15s 273ms/step - loss: 1.9738 - accuracy: 0.5330 - val_loss: 1.6143 - val_accuracy: 0.7394
Epoch 2/20
42/42 [==============================] - 10s 241ms/step - loss: 1.4589 - accuracy: 0.5816 - val_loss: 1.2742 - val_accuracy: 0.7394
Epoch 3/20
42/42 [==============================] - 10s 249ms/step - loss: 1.1313 - accuracy: 0.6529 - val_loss: 1.0676 - val_accuracy: 0.7394
Epoch 4/20
42/42 [==============================] - 10s 245ms/step - loss: 0.9384 - accuracy: 0.6916 - val_loss: 0.9380 - val_accuracy: 0.7394
Epoch 5/20
42/42 [==============================] - 10s 242ms/step - loss: 0.8131 - accuracy: 0.7174 - val_loss: 0.8489 - val_accuracy: 0.7394
Epoch 6/20
42/42 [==============================] - 11s 252ms/step - loss: 0.7341 - accuracy: 0.7231 - val_loss: 0.7881 - val_accuracy: 0.7394
Epoch 7/20
42/42 [==============================] - 10s 246ms/step - loss: 0.6788 - accuracy: 0.7291 - val_loss: 0.7462 - val_accuracy: 0.7394

In [192]:
# pad sequences to make them of the same length
maxlen = 2000  # max length of a text sequence
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# define model architecture
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=maxlen))
model.add(GRU(16, kernel_regularizer=regularizers.l2(0.001), recurrent_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# define early stopping callback
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3)

# train the model
model.fit(X_train, train_data['o'], validation_data=(X_val, val_data['o']), epochs=20, batch_size=64, callbacks=[early_stopping])

# evaluate the model on test data
loss, accuracy = model.evaluate(X_test, test_data['o'])
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/20
42/42 [==============================] - 66s 1s/step - loss: 0.8278 - accuracy: 0.6028 - val_loss: 0.7762 - val_accuracy: 0.7394
Epoch 2/20
42/42 [==============================] - 66s 2s/step - loss: 0.7541 - accuracy: 0.6859 - val_loss: 0.7110 - val_accuracy: 0.7394
Epoch 3/20
42/42 [==============================] - 65s 2s/step - loss: 0.7059 - accuracy: 0.6984 - val_loss: 0.6653 - val_accuracy: 0.7394
Epoch 4/20
42/42 [==============================] - 66s 2s/step - loss: 0.6716 - accuracy: 0.6988 - val_loss: 0.6345 - val_accuracy: 0.7394
Epoch 5/20
 7/42 [====>.........................] - ETA: 56s - loss: 0.6770 - accuracy: 0.6652

KeyboardInterrupt: 

In [184]:
y_pred=model.predict(X_test)

6/6 [==============================] - 1s 151ms/step


In [191]:
sum(y_pred>0.33)

array([165])

In [241]:
sum(train_data['o'])

794

In [6]:
y = df['c'].values

In [29]:
X_train, X_test, y_train, y_test= train_test_split(df1, y, test_size=0.2, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
ro=SMOTE()
X_train,y_train= ro.fit_resample(X_train,y_train)

In [47]:
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier
r=RandomForestClassifier(random_state=42,class_weight='balanced_subsample')
m=BaggingClassifier( random_state=42)

In [48]:
m.fit(X_train,y_train)

BaggingClassifier(random_state=42)

In [49]:
y_pred=m.predict(X_test)

In [50]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90       271
           1       0.50      0.29      0.37        59

    accuracy                           0.82       330
   macro avg       0.68      0.61      0.63       330
weighted avg       0.79      0.82      0.80       330



In [ ]:
              precision    recall  f1-score   support

           0       0.86      0.94      0.90       271
           1       0.50      0.29      0.37        59

    accuracy                           0.82       330
   macro avg       0.68      0.61      0.63       330
weighted avg       0.79      0.82      0.80       330


BaggingClassifier

In [ ]:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       247
           1       0.67      0.70      0.68        83

    accuracy                           0.84       330
   macro avg       0.78      0.79      0.79       330
weighted avg       0.84      0.84      0.84       330

m=RandomForestClassifier(class_weight='balanced_subsample',criterion='entropy',max_features='sqrt',n_estimators=1000,max_depth=170, random_state=42)
o